In [ ]:
# use only the first GPU
%env CUDA_VISIBLE_DEVICES=0

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

In [2]:
text, words_to_ids, ids_to_words, token_dict = pickle.load(open('data/preprocess.p', mode='rb'))

In [3]:
def pick_word(probabilities, int_to_vocab):
    return int_to_vocab[np.argmax(probabilities)]

In [8]:
gen_length = 500
checkout_dir = 'checkpoints/'
seq_length = 100
# homer_simpson, moe_szyslak, or Barney_Gumble
gen_sentences = ['homer','[space]','simpson','[colon]']
gen_sentences = ['bart','[space]','simpson','[colon]']
gen_sentences = ['moe','[space]','szyslak','[colon]']

tf.reset_default_graph()
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkout_dir + 'model.meta')
    loader.restore(sess, checkout_dir + 'model')

    # Get Tensors from loaded model
    names = ["input:0", "initial_state:0", "final_state:0", "probs:0"]
    tensors = [loaded_graph.get_tensor_by_name(name) for name in names]
    inputs, initial_state, final_state, probs = tensors

    # Sentences generation setup
    #gen_sentences = [prime_word]
    prev_state = sess.run(initial_state, {inputs: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[words_to_ids[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {inputs: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], ids_to_words)
        gen_sentences.append(pred_word)

## Remove tokens

In [7]:
tv_script = ''.join(gen_sentences)
for value, token in token_dict.items():
    tv_script = tv_script.replace(token, value)

print(tv_script)

bart simpson: (half under water) maggie! press the red button!

homer simpson: (annoyed grunt)

(springfield streets: ext. springfield streets - continuous)

nelson muntz: haw haw!

nelson muntz: haw haw!

nelson muntz: haw haw!

nelson muntz: haw haw!

c. montgomery burns: excellent.

lisa simpson: but...

bart simpson: i can't go write way! i just started seeing some nice guy and i want to know who you are. but right now she's  with my brother.

c. montgomery burns: (to homer) come on...

homer simpson: (determined) it's true. and i've got lots of potatoes (chuckles) what am i gonna do. get out. (then, looking around) where's the  of a cart or something or died?

homer simpson: (scared noise)

homer simpson: (scream)

homer simpson: (scream)

homer simpson: (pained noise)

homer simpson: (pained noise)

(fun center: ext. garden of eden - day)

homer simpson: (singing) when the time flew by to get it, i leave you / or is i should get a beer with my  a bag...

lisa simpson: (giving in 